In [1]:
import pandas as pd
import numpy as np
import re
import glob

Dammit Excel! Well, I now know that my first task is to clean up the files and get them into a working format. It appears as if there is a lot of superfluous information... someone was trying to impress a date with their excel talent. Let's create a function `make_df()` that will take a parameter to read in a sheet from my `orig_data` directory, extract the headers and apply those to the pertinent data. This is more complicated that what it sounds as they have taken the headers and split them among two rows. Fortunately they were consistent in their pattern so this function should work for the majority of the excel files that we will be working with.

# State Files

The following takes in one of the several files and cleans it up to make it in a workable format. There is still a lot to do, such as combining the dataframes by column headers.

In [2]:
def file_cleaner(sheet):
    # find all of the columns dealing with agency
    agency =pd.read_excel(sheet,skiprows=list(range(0,3)),usecols=list(range(0,9)))
    agency = agency.drop(agency.index[[0]])
    agency[['State', 'Agency Type']] = agency[['State', 'Agency Type']].fillna(method='ffill')
    agency = agency.reset_index(drop=True)
   
    # the find the rest of the columns
    attributes = pd.read_excel(sheet,skiprows=list(range(0,4)))
    attributes = attributes.ix[:,9:]
    
    # merge the two dataframes to be one
    merged = pd.concat([agency, attributes], axis=1)
    
    # clean up the headers in a very ugly way
    x = [y.replace('\n', '_') for y in merged.columns]
    z = [y.replace(' ', '_') for y in x]
    v = [y.replace('/', '_') for y in z]
    w = [y.replace('-_', '') for y in v]
    merged.columns = w
    return merged

In [6]:
sheets = glob.glob('orig_data/state_files/*')
df2012 = file_cleaner(sheets[0])
df2013 = file_cleaner(sheets[1])
df2014 = file_cleaner(sheets[2])
print(f.columns,f1.columns,f2.columns)

Index(['State', 'Agency_Type', 'Agency_Name', 'Population1', 'Total_Offenses',
       'Crimes_Against_Persons', 'Crimes_Against_Property',
       'Crimes_Against_Society', 'Crimes_Against_Persons',
       'Aggravated_Assault', 'Simple_Assault', 'Intimidation',
       'Homicide_Offenses', 'Murder_and_Nonnegligent_Manslaughter',
       'Negligent_Manslaughter', 'Justifiable_Homicide',
       'Kidnapping__Abduction', 'Sex_Offenses', 'Rape', 'Sodomy',
       'Sexual_Assault_With_An_Object', 'Fondling',
       'Sex_Offenses,_Nonforcible', 'Incest', 'Statutory_Rape', 'Arson',
       'Bribery', 'Burglary__Breaking_&_Entering', 'Counterfeiting__Forgery',
       'Destruction__Damage__Vandalism', 'Embezzlement',
       'Extortion__Blackmail', 'Fraud_Offenses',
       'False_Pretenses__Swindle__Confidence_Game',
       'Credit_Card__Automated_Teller_Machine_Fraud', 'Impersonation',
       'Welfare_Fraud', 'Wire_Fraud', 'LarcenyTheft_Offenses', 'Pocketpicking',
       'Pursesnatching', 'Shopliftin

In [7]:
df2012.to_csv('output_data/new_2012.csv')
df2013.to_csv('output_data/new_2013.csv')
df2014.to_csv('output_data/new_2014.csv')

Okay, so it appears that the column names don't match exactly. That is partly due to the 2012 file have less columns (need to look into this). But for 2013 and 2014 there appears to be the same information. Hmm... looks like superscripts are causing the problem.

In [138]:
for i in f2.columns:
    if i in f1.columns:
        print('TRUE')
    else:
        print(i)

TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
Human_Trafficking_Offenses
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
Sexual_Assault_With_an_Object
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE
TRUE


## Aggregated Files 

In [2]:
def make_df(sheet): 
    head_col = []
    
    file = pd.read_excel(sheet,header=None) # read in the file 
    head_col.extend(file.loc[3][0:2]) # extract the first two column headers and append it to a list
    file2 = pd.read_excel(sheet,header=None,skiprows=list(range(0,4))) # read in the file again as these rows weren't read in the first time, this time taking out the original rows
    head_col.extend(file2.loc[0][2:]) # append those to a list
    
    x = [y.replace('\n', '_') for y in head_col] # find all instances of \n in the file and replace with _
    z = [y.replace(' ', '_') for y in x ] # find all blanks and replace
    header_col = [x.lower() for x in z] # change the header column to lowercase
    
    fin_file = pd.read_excel(sheet,skiprows=list(range(0,7)),header=None) # read in the final file format
    fin_file.columns = header_col # attach the header column
    return fin_file

In [3]:
sheets = glob.glob('orig_data/*')
df = make_df(sheets[3])
df

,offense_category,total_victims1,10_and_under,11−15,16−20,21−25,26−30,31−35,36−40,41−45,46−50,51−55,56−60,61−65,66_and_over,unknown_age
0,Assault Offenses,1165909,41370,70960,131488,182801,157046,134489,103978,93937,81082,63908,40306,21419,24837,18288
1,Homicide Offenses,3854,243,56,417,632,472,401,324,280,263,206,166,118,221,55
2,Human Trafficking Offenses,57,0,19,27,4,2,1,2,0,1,0,0,0,0,1
3,Kidnapping/Abduction,15552,1810,1070,2135,2760,2230,1792,1087,847,662,399,255,144,210,151
4,Sex Offenses,70928,18731,17985,11890,6393,4279,3202,2135,1745,1490,1017,535,303,639,584
5,"Sex Offenses, Nonforcible",5484,715,3783,864,35,18,10,11,7,4,7,1,1,3,25
6,Crimes Against Property,2770816,9172,29432,168102,313010,296920,280929,244772,248560,249127,238436,204616,154622,280975,52143
7,Arson,10614,253,133,365,809,901,965,891,924,997,1006,814,590,1046,920
8,Bribery,222,2,7,22,21,45,27,21,17,16,7,5,3,4,25
9,Burglary/Breaking & Entering,447058,1669,2399,19496,45909,46353,45528,39720,39752,40719,40554,35699,28183,53629,7448


Looks like it didn't work for all tables. Seems as if there are some files that start with only 2 unnecessary columns. I will need to see if I can clean this up so that I don't have two functions doing the same thing.

In [4]:
def make_df(sheet): 
    head_col = []
    
    file = pd.read_excel(sheet,header=None) # read in the file 
    head_col.extend(file.loc[2][0:2]) # extract the first two column headers and append it to a list
    file2 = pd.read_excel(sheet,header=None,skiprows=list(range(0,3))) # read in the file again as these rows weren't read in the first time, this time taking out the original rows
    head_col.extend(file2.loc[0][2:]) # append those to a list
    
    x = [y.replace('\n', '_') for y in head_col] # find all instances of \n in the file and replace with _
    z = [y.replace(' ', '_') for y in x ] # find all blanks and replace
    header_col = [x.lower() for x in z] # change the header column to lowercase
    
    fin_file = pd.read_excel(sheet,skiprows=list(range(0,7)),header=None) # read in the final file format
    fin_file.columns = header_col # attach the header column
    return fin_file

In [5]:
df = make_df(sheets[1])
df

,offense_category,total_victims1,family_member2,family_member_and_other3,known_to_victim_and_other4,stranger,all_other5
0,Homicide Offenses,3427,602,46,1317,391,1071
1,Human Trafficking Offenses,57,0,0,28,24,5
2,Kidnapping/Abduction,14894,3226,179,7969,1631,1889
3,Sex Offenses,66567,17770,607,35497,4911,7782
4,"Sex Offenses, Nonforcible",5191,1066,52,3624,95,354
5,Crimes Against Property,64357,569,170,12431,27370,23817
6,Robbery,64357,569,170,12431,27370,23817
7,1The relationship of a victim to an offender i...,NaN,NaN,NaN,NaN,NaN,NaN
8,"2Victims were related to all offenders, whethe...",NaN,NaN,NaN,NaN,NaN,NaN
9,3Victims were related to at least one of the m...,NaN,NaN,NaN,NaN,NaN,NaN


Soooooo, I am going to go in a different direction temporarily. It turns out there is a lot more information out in there and that inforation is more appealing than that information. Swictching gears. Okay, this site also includes types of offenses by cities, so first things first, lets build some tables to feed into a database. 

**STATES FIRST**

In [7]:
state = pd.read_excel('orig_data/state_files/United_States_Offense_Type_by_Agency_2014.xls',skiprows=list(range(0,3)), usecols = ['State']).dropna(axis =0)
states = state.drop(state.index[[38,39,40]])

In [8]:
states

,State
1,ALABAMA
2,ARIZONA
6,ARKANSAS
257,COLORADO
465,CONNECTICUT
548,DELAWARE
598,DISTRICT OF COLUMBIA
599,IDAHO
705,ILLINOIS
706,INDIANA


**AGENCIES SECOND**

In [9]:
agency =pd.read_excel('orig_data/state_files/United_States_Offense_Type_by_Agency_2014.xls',skiprows=list(range(0,3)),usecols=['State','Agency Type','Agency Name','Population1'])

In [10]:
agency = agency.drop(agency.index[[0]])

In [11]:
agency = agency.fillna(method='ffill')


In [54]:
agencies = agency.drop(agency.index[[5454,5455,5456]])
agencies

,State,Agency Type,Agency Name,Population1
1,ALABAMA,Cities,Hoover,84843
2,ARIZONA,Cities,Apache Junction,37274
3,ARIZONA,Cities,Gilbert,235430
4,ARIZONA,Cities,Yuma,92075
5,ARIZONA,Metropolitan Counties,Yuma,92075
6,ARKANSAS,Cities,Alma,5535
7,ARKANSAS,Cities,Altus,742
8,ARKANSAS,Cities,Arkadelphia,10721
9,ARKANSAS,Cities,Ashdown,4552
10,ARKANSAS,Cities,Ash Flat,1072
